In [1]:
import torch as tc
from torch.autograd import Variable

# Sementic analysis and word embeddings

Here, we investigate **semantic similarity** based on the **distributional hypothesis**: words appearing in similar contexts are related to each other semantically.


## Word embeddings

Word embeddings are a **representation of the semantics of a word**: efficiently **encoding semantic information** that may be relavant to the task at hand.

Embeddings are stored as a **$|V|×D$ matrix**, where $D$ is the dimensionality of the embeddings, such that the word assigned index $i$ has its embedding stored in the $i$‘th row of the matrix. 

In [4]:
tc.manual_seed(1)

# Mapping from word to index
word_to_ix = {"hello": 0, "world": 1}

embeds = tc.nn.Embedding(2, 5)
lookup_tensor = tc.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(Variable(lookup_tensor))
print(hello_embed)

Variable containing:
 0.6614  0.2669  0.0617  0.6213 -0.4519
[torch.FloatTensor of size 1x5]



In [170]:
list(embeds.parameters())

[Parameter containing:
  0.6614  0.2669  0.0617  0.6213 -0.4519
 -0.1661 -1.5228  0.3817 -1.0276 -0.5631
 [torch.FloatTensor of size 2x5]]

## N-Gram Language modelling

In [141]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [15]:
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence)-2)]

In [20]:
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [171]:
vocab = set(test_sentence)
word_to_ix = {word: idx for idx, word in enumerate(vocab)}

In [145]:
class NGramLanguageModeler(tc.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embedding = tc.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = tc.nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = tc.nn.Linear(128, vocab_size)
        
    def forward(self, x):
        embeds = self.embedding(x).view((1, -1)) # shape: 1, 20
        # print("embeds.shape: ", embeds.shape)
        out = tc.nn.functional.relu(self.linear1(embeds)) # shape: 1, 128
        # print("out1.shape: ", out.shape)
        out = self.linear2(out) # shape: 1, 97
        # print("out2.shape: ", out.shape)
        log_probs = tc.nn.functional.log_softmax(out, dim=1)
        return log_probs
    
losses = []
criterion = tc.nn.NLLLoss()

EMBEDDING_DIM = 10
CONTEXT_SIZE = 2


model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = tc.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = tc.Tensor([0])
    for context, target in trigrams:
        context_idx = [word_to_ix[w] for w in context]
        context_var = Variable(tc.LongTensor(context_idx))
        
        model.zero_grad()
        log_probs = model(context_var)
        
        loss = criterion(log_probs, Variable(tc.LongTensor([word_to_ix[target]])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
    if epoch%50 == 0:
        print("epoch: {}".format(epoch))
print(losses)

Variable containing:
 84
 21
[torch.LongTensor of size 2]

Variable containing:
 21
 78
[torch.LongTensor of size 2]

Variable containing:
 78
  2
[torch.LongTensor of size 2]

Variable containing:
  2
 48
[torch.LongTensor of size 2]

Variable containing:
 48
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 32
[torch.LongTensor of size 2]

Variable containing:
 32
 22
[torch.LongTensor of size 2]

Variable containing:
 22
 29
[torch.LongTensor of size 2]

Variable containing:
 29
 27
[torch.LongTensor of size 2]

Variable containing:
 27
 44
[torch.LongTensor of size 2]

Variable containing:
 44
  0
[torch.LongTensor of size 2]

Variable containing:
  0
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 39
[torch.LongTensor of size 2]

Variable containing:
 39
 40
[torch.LongTensor of size 2]

Variable containing:
 40
 17
[torch.LongTensor of size 2]

Variable containing:
 17
 83
[torch.LongTensor of size 2]

Variable containing:
 83
  7
[torch.LongTensor of size 2

Variable containing:
 36
 12
[torch.LongTensor of size 2]

Variable containing:
 12
 72
[torch.LongTensor of size 2]

Variable containing:
 72
 24
[torch.LongTensor of size 2]

Variable containing:
 24
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 47
[torch.LongTensor of size 2]

Variable containing:
 47
  3
[torch.LongTensor of size 2]

Variable containing:
  3
 25
[torch.LongTensor of size 2]

Variable containing:
 25
 91
[torch.LongTensor of size 2]

Variable containing:
 91
 64
[torch.LongTensor of size 2]

Variable containing:
 64
  4
[torch.LongTensor of size 2]

Variable containing:
  4
 41
[torch.LongTensor of size 2]

Variable containing:
 41
 27
[torch.LongTensor of size 2]

Variable containing:
 27
 60
[torch.LongTensor of size 2]

Variable containing:
 60
 50
[torch.LongTensor of size 2]

Variable containing:
 50
 16
[torch.LongTensor of size 2]

Variable containing:
 16
 33
[torch.LongTensor of size 2]

Variable containing:
 33
 34
[torch.LongTensor of size 2

Variable containing:
 38
 11
[torch.LongTensor of size 2]

Variable containing:
 11
 58
[torch.LongTensor of size 2]

Variable containing:
 58
 80
[torch.LongTensor of size 2]

Variable containing:
 80
 54
[torch.LongTensor of size 2]

Variable containing:
 54
 37
[torch.LongTensor of size 2]

Variable containing:
 37
  8
[torch.LongTensor of size 2]

Variable containing:
  8
 36
[torch.LongTensor of size 2]

Variable containing:
 36
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 76
[torch.LongTensor of size 2]

Variable containing:
 76
 59
[torch.LongTensor of size 2]

Variable containing:
 59
 71
[torch.LongTensor of size 2]

Variable containing:
 71
 36
[torch.LongTensor of size 2]

Variable containing:
 36
 12
[torch.LongTensor of size 2]

Variable containing:
 12
 72
[torch.LongTensor of size 2]

Variable containing:
 72
 24
[torch.LongTensor of size 2]

Variable containing:
 24
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 47
[torch.LongTensor of size 2

Variable containing:
 50
 16
[torch.LongTensor of size 2]

Variable containing:
 16
 33
[torch.LongTensor of size 2]

Variable containing:
 33
 34
[torch.LongTensor of size 2]

Variable containing:
 34
  6
[torch.LongTensor of size 2]

Variable containing:
  6
 82
[torch.LongTensor of size 2]

Variable containing:
 82
 90
[torch.LongTensor of size 2]

Variable containing:
 90
 53
[torch.LongTensor of size 2]

Variable containing:
 53
 88
[torch.LongTensor of size 2]

Variable containing:
 88
 96
[torch.LongTensor of size 2]

Variable containing:
 96
 28
[torch.LongTensor of size 2]

Variable containing:
 28
 30
[torch.LongTensor of size 2]

Variable containing:
 30
 63
[torch.LongTensor of size 2]

Variable containing:
 63
 79
[torch.LongTensor of size 2]

Variable containing:
 79
 39
[torch.LongTensor of size 2]

Variable containing:
 39
 86
[torch.LongTensor of size 2]

Variable containing:
 86
 31
[torch.LongTensor of size 2]

Variable containing:
 31
 66
[torch.LongTensor of size 2


Variable containing:
 55
 56
[torch.LongTensor of size 2]

Variable containing:
 56
 93
[torch.LongTensor of size 2]

Variable containing:
 93
 26
[torch.LongTensor of size 2]

Variable containing:
 26
  1
[torch.LongTensor of size 2]

Variable containing:
  1
 82
[torch.LongTensor of size 2]

Variable containing:
 82
 20
[torch.LongTensor of size 2]

Variable containing:
 20
 26
[torch.LongTensor of size 2]

Variable containing:
 26
 62
[torch.LongTensor of size 2]

Variable containing:
 62
 85
[torch.LongTensor of size 2]

Variable containing:
 85
  9
[torch.LongTensor of size 2]

Variable containing:
  9
 52
[torch.LongTensor of size 2]

Variable containing:
 52
 76
[torch.LongTensor of size 2]

Variable containing:
 76
 13
[torch.LongTensor of size 2]

Variable containing:
 13
 45
[torch.LongTensor of size 2]

Variable containing:
 45
 89
[torch.LongTensor of size 2]

Variable containing:
 89
 77
[torch.LongTensor of size 2]

Variable containing:
 77
 51
[torch.LongTensor of size 

In [135]:
print(list(model.parameters())[0].shape) # Shape of word embedding
print(list(model.parameters())[1].shape) # Shape of the weight for self.linear1
print(list(model.parameters())[2].shape) # Shape of the bias for self.linear1
print(list(model.parameters())[3].shape) # Shape of the weight for self.linear2
print(list(model.parameters())[4].shape) # Shape of the bias for self.linear2

torch.Size([97, 10])
torch.Size([256, 20])
torch.Size([256])
torch.Size([97, 256])
torch.Size([97])


# Computing word embeddings: Continuous Bag-of-Words

The CBOW model is as follows: Given a target word $w_i$ and an $N$ context window on each side, $w_{i−1},\dots,w{i−N}$ and $w_{i+1},…,w_{i+N}$, referring to all context words collectively as $C$, CBOW tries to minimize

$$−\log p(w_i|C) = −\log\text{Softmax}(A(\sum_{w\in C}q_w)+b)$$
where $q_w$ is the embedding for word $w$.

For this exercise, let's us the following text as the test sentence:

In [101]:
test_sentence = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

Now we create continuous Bag-of-Words

In [107]:
def n_words_before_i(i, n):
    reverse_indices = list(range(1,n+1))
    reverse_indices.reverse()
    return [test_sentence[i-k] for k in reverse_indices]

def n_words_after_i(i, n):
    indices = range(1,n+1)
    return [test_sentence[i+k] for k in indices]

In [108]:
n_window = 5
Continuous_BOW = [[n_words_before_i(i, n_window), test_sentence[i], n_words_after_i(i, n_window)] for i in range(n_window, len(test_sentence) - n_window)]

In [109]:
Continuous_BOW

[[['We', 'are', 'about', 'to', 'study'],
  'the',
  ['idea', 'of', 'a', 'computational', 'process.']],
 [['are', 'about', 'to', 'study', 'the'],
  'idea',
  ['of', 'a', 'computational', 'process.', 'Computational']],
 [['about', 'to', 'study', 'the', 'idea'],
  'of',
  ['a', 'computational', 'process.', 'Computational', 'processes']],
 [['to', 'study', 'the', 'idea', 'of'],
  'a',
  ['computational', 'process.', 'Computational', 'processes', 'are']],
 [['study', 'the', 'idea', 'of', 'a'],
  'computational',
  ['process.', 'Computational', 'processes', 'are', 'abstract']],
 [['the', 'idea', 'of', 'a', 'computational'],
  'process.',
  ['Computational', 'processes', 'are', 'abstract', 'beings']],
 [['idea', 'of', 'a', 'computational', 'process.'],
  'Computational',
  ['processes', 'are', 'abstract', 'beings', 'that']],
 [['of', 'a', 'computational', 'process.', 'Computational'],
  'processes',
  ['are', 'abstract', 'beings', 'that', 'inhabit']],
 [['a', 'computational', 'process.', 'Com

In [103]:
class CBOW(tc.nn.Module):
    def __init__(self, vocab_len, embedding_len, context_len):
        super(CBOW, self).__init__()
        self.embed = tc.nn.Embedding(vocab_len, embedding_len)
        self.linear2 = tc.nn.Linear(context_len*embedding_len, vocab_len)
        
    def forward(self, x):
        x = self.embed(x).view((1, -1))
        return tc.nn.functional.log_softmax(self.linear2(x))

In [111]:
vocab = set(test_sentence)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

len_context = 5
len_embedding = 10
len_vocab = len(vocab)

criterion = tc.nn.NLLLoss()
model = CBOW(len_vocab, len_embedding, 2*len_context)
optimizer = tc.optim.SGD(model.parameters(), lr=0.001)

In [112]:
losses = []
for epoch in range(5):
    total_loss = tc.Tensor([0])
    for context1, word, context2 in Continuous_BOW:
        context_word = [word_to_idx[w] for w in context1 + context2]
        context_var = Variable(tc.LongTensor(context_word))
        
        word = Variable(tc.LongTensor([word_to_idx[word]]))

        optimizer.zero_grad()
        pred_word = model(context_var)
        loss = criterion(pred_word, word)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 209.7036
[torch.FloatTensor of size 1]
, 
 204.3476
[torch.FloatTensor of size 1]
, 
 199.0314
[torch.FloatTensor of size 1]
, 
 193.7571
[torch.FloatTensor of size 1]
, 
 188.5271
[torch.FloatTensor of size 1]
]


/home/sewook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
